In [1]:
from collections import defaultdict
import jsonlines
# get_gpt_generation() function from utils
from utils import * 

In [11]:
from src.machine_translation import *
import evaluate
chrf = evaluate.load("chrf")

# Prompting

In [3]:

def machine_translation(test_examples):
    answers = defaultdict(list)
    
    for ex in test_examples:
        to_eng = ex['to_eng']
        to_oth = ex['to_oth']
    
        eng = ex['eng']
        oth = ex['oth']
        
        ###### HERE YOU CAN CHANGE IT TO YOUR OWN MODELS########
        gen_to_eng = get_gpt_generation(to_eng)
        gen_to_oth = get_gpt_generation(to_oth)
        ########################################################
    
        ex['gen_to_eng']= gen_to_eng
        ex['gen_to_oth']= gen_to_oth
    
        with jsonlines.open('results/machine_translation.jsonl', mode='a') as writer:
            writer.write(ex) 

In [ ]:
test_examples, test_ids = flores_mt()
machine_translation(test_examples)

# Scoring

In [ ]:
pairs = ['eng_Latn-zho_Hans',
 'eng_Latn-kor_Hang',
 'eng_Latn-jpn_Jpan',
 'eng_Latn-fra_Latn',
 'eng_Latn-spa_Latn',
 'eng_Latn-ind_Latn',
 'eng_Latn-vie_Latn',
 'eng_Latn-arb_Arab',
 'eng_Latn-jav_Latn',
 'eng_Latn-sun_Latn',
 'eng_Latn-bug_Latn']

In [8]:
objs = []
with jsonlines.open('machine_translation.jsonl') as read_file:
    for line in read_file.iter():
    	objs.append(line)

lang_per_result = defaultdict(list)
for obj in objs:
    lang_per_result[obj['lang_pair']].append(obj)

In [12]:
results_to_eng = {}
results_to_oth = {}

for target in pairs:
    
    prediction_to_eng = [x['gen_to_eng'] for x in lang_per_result[target]]
    ref_to_eng = [[x['eng']] for x in lang_per_result[target]]
    results_to_eng[target] = chrf.compute(predictions=prediction_to_eng, references=ref_to_eng)['score']
    
    prediction_to_oth = [x['gen_to_oth'] for x in lang_per_result[target]]
    ref_to_oth = [[x['oth']] for x in lang_per_result[target]]
    results_to_oth[target] = chrf.compute(predictions=prediction_to_oth, references=ref_to_oth)['score']


dict_keys(['id', 'lang_pair', 'domain', 'topic', 'eng', 'oth', 'to_eng', 'to_eng_ref', 'to_oth', 'to_oth_ref', 'gen_to_eng', 'gen_to_oth'])


## Read results

In [14]:
high = ['eng_Latn-zho_Hans',
 'eng_Latn-kor_Hang',
 'eng_Latn-jpn_Jpan',
 'eng_Latn-fra_Latn',
 'eng_Latn-spa_Latn',
 'eng_Latn-ind_Latn',
 'eng_Latn-vie_Latn',
 'eng_Latn-arb_Arab']

low = [ 'eng_Latn-jav_Latn',
 'eng_Latn-sun_Latn',
 'eng_Latn-bug_Latn']
    

In [19]:
result_high_to_eng = 0
for h in high:
    result_high_to_eng+=results_to_eng[h]
print("High: xxx->eng || ", round(result_high_to_eng/len(high),3))

result_low_to_eng = 0
for l in low:
    result_low_to_eng+=results_to_eng[l]
print(r"Low: xxx->eng || ", round(result_low_to_eng/len(low), 3))

High: xxx->eng ||  63.247
Low: xxx->eng ||  39.026


In [20]:
result_high_to_oth = 0
for h in high:
    result_high_to_oth+=results_to_oth[h]
print("High: eng->xxx || ", round(result_high_to_oth/len(high),3))

result_low_to_oth = 0
for l in low:
    result_low_to_oth+=results_to_oth[l]
print(r"Low: eng->xxx || ", round(result_low_to_oth/len(low), 3))

High: eng->xxx ||  55.027
Low: eng->xxx ||  37.926
